<a id ='top'></a>

# Most important functions:

- Download ids (or ids + info) of a Twitter page followers
- Download the pages liked by a certain user
- Eliminate all duplicates ids of users who follow a certain page


Most functions can be called at the same time, as they don't overlap; for example, one can create a copy of the notebook and download ids in one and pages liked in the other. While doing so, though, it is ideal to start the call in the second notebook as the first one is sleeping, or to use different databases, in order to avoid occasional locked database errors (OperationalError: database is locked).



We built 2 different versions of both get_followers and get_followers_ids, solving different needs. The smartest implementation overall is the *faster version of get_followers*, where we check the tweepy_page in order to stop downloading followers when we reach the last page (very efficient and effective). It is a thought we had only after a while, so other functions could gain in efficiency by being adapted to use this same method


* [Tweepy rate limits](https://developer.twitter.com/en/docs/basics/rate-limits)
* [Tweepy count limits](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-list)
* [Tweepy documentation](http://docs.tweepy.org/en/latest/)


# Index:
    
- [Libraries, Database and general functions](#Funzioni) (run the first two cells)
- [Get Followers](#get_followers);   &nbsp;&nbsp;&nbsp;&nbsp;[faster version](#faster_get_followers)
- [Get Followers ids](#get_followers_ids);     &nbsp;&nbsp;&nbsp;&nbsp;[faster version](#faster)
- [Get Pages liked by an user](#get_pages)
- [Dataset cleaning and viewing](#dataset)


## DB Tables


description of the tables in our database (note: we left only information related to 'lavazzagroup' to subset it):
- _Users_  
>  contains users who follow a page along with other info on them; 
>
>
> columns: ['twitter_page', 'user_id', 'description', 'location', 'status_count', 'follow_count', 'friend_count', 'changed_img', 'changed_theme', 'pages_liked']

- _Cursor2_ 
>  contains the tweepy page at which we arrived using get_followers on a twitter page;

- _twitter_data_ 
> contains only the ids of people who follow a brand;
>
>
>columns = ['brand', 'followerid']

- _Cursor_ 
>  contains the tweepy page at which we arrived using get_followers_ids on a twitter page;


<a id ='Funzioni'></a>

# Libraries, database, general functions

### <font color = #ff9900 >Please modify the path! </font>

It is necessary to run the following two cells

In [1]:
import os
import json 

# path
b_dir= r'C:/Users/Eugen/Documents/Uni/1 Marketing/API stuff' 
# load the auth file with all the apps
c_auths = json.load(open(os.path.join(b_dir)+'/chain_au.txt'))

import tweepy
import pandas as pd
import sqlite3
import time
import numpy as np

# define the function we will use to connect to the db. It can also be used to get the df or create
# a new table with specific columns
def connect_to_sql(database_name = 'Brand_followers.db', 
                   table_name = 'twitter_data', 
                   column_names = ['brand', 'followerid'],
                   create_table = False,
                   return_df = False):
    
    con = sqlite3.connect(database_name) #create or connect to the db
    cur = con.cursor()
    
    if create_table == True:
        # generating a string out of the column names
        str_column_names = '('
        for i in column_names:
            str_column_names += str(i) + ','
        str_column_names = str_column_names[:-1] + ')'
        cur.execute("CREATE TABLE IF NOT EXISTS "+ table_name + str_column_names)
    
    if return_df == True:
        query = "SELECT * FROM " + table_name    
        df = pd.read_sql_query(query, con)
        return con, cur, df
    else:
        return con, cur
    
    
# define the db to connect to
database_to_use = 'Brand_Followers.db'

# connect to the db
con, cur =     connect_to_sql(database_name = database_to_use, 
                              return_df = False)  

# print table names
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables in the db:", cur.fetchall()) 

Tables in the db: []


In [2]:
def authenticate( wait_T_or_F = True,     b_dir= r'C:/Users/Eugen/Documents/Uni/1 Marketing/API stuff' ):
    '''   
    Purpose: authenticate on twitter and create the object connected to tweepy
    
    Arguments: wait_T_or_F = whether to wait on rate limit
    '''
    
    auths = json.load(open(os.path.join(b_dir)+'/au.txt'))
    auth = tweepy.OAuthHandler(auths[0], auths[1])
    auth.set_access_token(auths[2], auths[3]) 
    api=tweepy.API(auth, 
                   wait_on_rate_limit        = wait_T_or_F, 
                   wait_on_rate_limit_notify = wait_T_or_F)
    
    return api 


def c_authenticate(c_auths, app_num, wait_T_or_F = True):
    '''
     Purpose: same as authenticate, but allows to login in a specific app among a list of apps
     
     Arguments: c_auths: .txt file with a list of apps credentials (list of lists, as app
                         credentials are stored in a list)
                app_num: the app to use 

    '''   
            
    auth = tweepy.OAuthHandler(c_auths[app_num][0], c_auths[app_num][1])
    auth.set_access_token(c_auths[app_num][2], c_auths[app_num][3]) 
    #print(c_auths[app_num][0])
    api = tweepy.API(auth, wait_on_rate_limit= wait_T_or_F, wait_on_rate_limit_notify = wait_T_or_F)

    return api
    

def rem_duplicates_if(df, 
                      page_of_interest,
                      column_w_page_name = 'twitter_page',
                      column_with_duplicates =  'user_id'):
    '''    
    Purpose: returns the df subset such that df[df[column_of_interest] == page_of_interest], 
    eliminating duplicates;
    e.g. all the ids of the followers of page_of_interest = 'lavazzagroup', without duplicate ids
    
    Arguments:  page_of_interest: the page whose followes ids we want, without duplicates
                table_name: the name of the sql table to use
                column_w_page_name: column of the table in which twitter pages are stored
                column_with_duplicates: column containing followers ids (or, in general, the values
                which we want to be returned, avoiding duplicates)


    '''
    if len(df) == 0:
        return df
    
    df = df[df[column_w_page_name] == page_of_interest]  
    df_no_duplicates = df[[(lambda l: not bool(l))(item) for item in df.duplicated(subset=column_with_duplicates, keep='first')]]
    
    return df_no_duplicates

def rem_duplicates_dataset(df, 
                           column_w_page_name = 'brand',
                           column_with_duplicates =  'followerid'):
    '''
    Purpose: removes duplicate followers for each page, returning the whole dataset.
             It works operating page by page, or we would remove also people who follow
             several different pages stored in our dataset
    '''
    if len(df) == 0:
        print("invalid df")
        return df
    print("Initial shape", df.shape)  
    
    # creating array with twitter pages 
    twitter_pages = df[column_w_page_name].unique()

    # use rem_duplicates_if on each twitter page, collecting the dfs and concatenating them at the end
    temp_list = []
    for twitter_page in twitter_pages:
        df_clean = rem_duplicates_if(df, page_of_interest = twitter_page, column_w_page_name = column_w_page_name, column_with_duplicates=column_with_duplicates)
        temp_list.append(df_clean)

    df_final = pd.concat(temp_list)
    print("Final shape", df_final.shape)  
    return df_final
           
    
def num_unique_followers(table_name, column_w_page_name, column_with_duplicates, page_of_interest):
    '''
    Purpose: calculate the number of unique followers we have for a twitter page,
             given an sql table, the columns and the twitter page 
    '''
    query = "SELECT * FROM {} where {} = '{}'".format(table_name, column_w_page_name, page_of_interest)
    df2 = pd.read_sql_query(query, con)
    return len(df2["{}".format(column_with_duplicates)].unique())               

<a id ='get_followers'></a>

# Get Followers
[Back on top](#top)

In [11]:
def get_followers   (twitter_page: str ,
                     api,
                     column_with_duplicates =  'user_id',
                     column_w_page_name = 'twitter_page',
                     table_name = 'Users',
                     *prior_info # insert num_ids_down
                    ):
    '''
    Purpose:    Similar to get_followers_ids, but also gets other information in the process,
                so it operates at a slower rate.
                Opposed to get_followers_ids, it doesn't automatically iterate, and, by default,  
                it stores in a different table ('Users' instead of 'twitter_data')
                
    Argomenti:  twitter_page = page whose users ids we want to retrieve
                api = the object connected to tweepy
                table_name = db table in which to store the data
                column_with_duplicates = column on which we sanity check to be sure we aren't 
                                         downloading duplicates ('user_id')
                column_w_page_name = column which contains the twitter pages names
                prior_info: bool on whether we already downloaded some users of that page,
                            and an int on how many we downloaded (provide 0 if the bool is False)
    '''
    
    # tot num of follw of the page
    follw_count = api.get_user(twitter_page).followers_count  # tot num of follw of a page
    # max num of downloadable user objects per tweepy page
    id_downl_at_step = 200
    
# access/create the db containing the cursors values, will be useful 
# to know at which tweepy page to start or resume the download
    cur.execute("CREATE TABLE IF NOT EXISTS Cursor2(twitter_page, cursor_value)")
    
    # if the page is new, start from scratch, else from the cur val in the db
    try:
        # if we haven't this ids already, it will give an error, prompting the except
        query_2 = "SELECT cursor_value FROM Cursor2 WHERE twitter_page = '{}'".format(twitter_page)              
        num_page = int(pd.read_sql_query(query_2, con).loc[0])
        if not prior_info:
            num_ids_downloaded = num_unique_followers(table_name, column_w_page_name, column_with_duplicates, twitter_page)
        else:
            num_ids_downloaded = prior_info[0]
            
    # if we have no ids or no cursor value, start from scratch
    except:
        num_page = -1
        num_ids_downloaded = 0
    
    print("\n", twitter_page, "has {} followers, we already have {} ids".format(follw_count, num_ids_downloaded))    
       
    bag = []        
    # get follower ids and other info on them
    tweepy_cursor = tweepy.Cursor(api.followers, id= twitter_page, count = id_downl_at_step, cursor = num_page).pages(15)
    for result_set in tweepy_cursor:
        for user in result_set:
            bag.append([twitter_page, user.id, user.description, user.location, user.statuses_count, user.followers_count,
                      user.friends_count, user.default_profile_image, user.default_profile, np.nan])
    # update cursor value
    num_page = tweepy_cursor.next_cursor
    
    # store the info in the db. To be made more efficient, can put this step outside
    # the function, storging less frequently
    qst_marks = "?,"*len(bag[0])
    cur.executemany("INSERT INTO {} VALUES ({})".format(table_name, qst_marks[:-1]), bag) 
    
    # if the page is new, create a value in the db, else update it
    if num_ids_downloaded == 0: 
        cur.execute("INSERT INTO Cursor2 (twitter_page, cursor_value) VALUES (?,?)", [twitter_page, num_page])
    else: 
        cur.execute("UPDATE Cursor2 SET cursor_value = " + str(num_page) + " WHERE twitter_page ='{}'".format(twitter_page))
        
    con.commit()
    
    # check if we really downloaded new data    
    final_ids = num_unique_followers(table_name, column_w_page_name, column_with_duplicates, twitter_page)
    print("df final length = {}, \ndf initial length = {}, \nnew ids added: {}".format(final_ids, num_ids_downloaded, final_ids-num_ids_downloaded))
    
    # return the final number of unique ids we have
    return final_ids


def get_user(user_id, api, table_name = 'Users', store = True):   # up to 900 requests each 15 minutes
    '''
    WHAT DOES IT DO?
    Collect other info on a user, useful if we only have the id. Stores it in the Users table
    '''
    try:
        if type(user_id) == int:
            user = api.get_user(user_id)
            bag = [user.description, user.location, user.statuses_count, user.followers_count,
                                  user.friends_count, user.default_profile_image, user.default_profile, user_id]


        else: 
            bag = []
            for single_user_id in user_id:
                user = api.get_user(single_user_id)
                bag.append((user.description, user.location, user.statuses_count, user.followers_count,
                          user.friends_count, user.default_profile_image, user.default_profile, single_user_id))
                
        if store == True:
            cur.executemany("UPDATE "+table_name+" SET description =?, location =?, status_count =?, follow_count =?, friend_count =?, changed_img =?, changed_theme =? WHERE user_id = ?",  bag)
            con.commit()
                
        return bag
        
    except:
        print("Twitter error for this user")
        return np.nan

    
    
    
def get_followers_chained_all_stats(twitter_page,
                                    follw_count,
                                    table_name = 'Users',
                                    column_w_page_name = 'twitter_page',
                                    column_with_duplicates = 'user_id',
                                    b_dir= r'C:/Users/Eugen/Documents/Uni/1 Marketing/API stuff'):
    '''
    Purpose: unless we generate a number of apps, we will have sleeping times.
             To make the most of them, chain get_followers with get_followers_ids
             and user methods to be able to download more ids and info on users
             every 15 minutes. These calls, despite being of the same type (GET),
             don't interfere with each other's rate limits
    '''

    # load the file with the auths
    c_auths = json.load(open(os.path.join(b_dir)+'/chain_au.txt'))
    
    num_ids_downloaded = num_unique_followers(table_name, column_w_page_name, column_with_duplicates, twitter_page)
    
    print("\n", twitter_page, "has {} followers, we already have {} ids".format(follw_count, num_ids_downloaded))
    
    # at each iter we get 3200 ids and info, per each app used
    num_iterations = int(np.ceil( (follw_count - num_ids_downloaded) / (3200* len(c_auths)) ))
    time_check = False
    
    # at each iteration we go through all apps and call different methods
    for iteration in range(0, num_iterations):
        print("We are at iteration {} on {}".format(iteration + 1, num_iterations))
        
        # except the first time, sleep if needed
        if time_check == True:     
            time_elapsed = time.time() - start_time 
            if time_elapsed < (15*60) + 5:
                print("So far {} seconds have passed, need to sleep for {}".format(time_elapsed, 60*15 - time_elapsed))
                time.sleep(time_elapsed)

        # switching auths
        for app_num in range(len(c_auths)):
            print("Using App number: ", app_num)
            api = c_authenticate(c_auths, app_num)

           # starting to retrieve data, inizializing sleep stuff
            print("\nGETTING FOLLOWERS")  
            if app_num == 0:
                start_time = time.time()

            num_ids_downloaded = get_followers(twitter_page, # gets 3000 ids and stats
                                               api,
                                               column_with_duplicates,
                                               column_w_page_name,
                                               table_name,
                                               num_ids_downloaded) 
            time_check = True
            
            if num_ids_downloaded >= follw_count*0.97:
                print("\n We already have all ids; terminating the call")
                print("ids we have:", str(num_ids_downloaded), "ids of the page: ", str(follw_count) )
                return #breaks all loops

            print("\nGETTING FOLLOWERS IDS")
            
            # luckily, we can use the same cursor we created above
            try:
                query_2 = "SELECT cursor_value FROM Cursor2 WHERE twitter_page = '{}'".format(twitter_page)  
                num_page = int(pd.read_sql_query(query_2, con).loc[0])
                
            # added an except in case the above part got skipped for any reason, and
            # we have no ids (thus no cursor value) for this page
            except:
                num_page= -1
                
            user_ids = []
            # based on the speed of computer and connection, we can get multiples of 
            # 200 ids at this step. Chaining through 5 apps, it leads to multiples of 1000
            # more ids and info every 15 mins (up to a maxiumum of 8000, due to get_user rate
            # limit)
            tweepy_cursor = tweepy.Cursor(api.followers_ids, id = twitter_page, cursor = num_page, count = 200 ).pages(1) 
            for id_list in tweepy_cursor:
                user_ids.extend(id_list)
            num_page = tweepy_cursor.next_cursor
                        
            g = list(zip([twitter_page]*len(user_ids), user_ids))

            cur.executemany("INSERT INTO {}(twitter_page, user_id) VALUES (?,?)".format(table_name), (g)) 
            cur.execute("UPDATE Cursor2 SET cursor_value = " + str(num_page) + " WHERE twitter_page ='{}'".format(twitter_page))
            con.commit()
            
            print("\nGETTING STATS")
            '''
            was used initially to get also some liked pages, but not worth it much
            
            print("\nGETTING STATS AND PAGES LIKED BY IDS")
            #i = 0
            id_page_list = []
            user_info = []
            for user_id in user_ids: 
                
                #get some of the pages liked by the first 15 users in the list
                #if i < 15:
                #    i += 1
                #    pages = get_pages(_id = user_id, api = api)
                #    id_page_list.append((user_id, pages))
                #    if i == 14:
                #        store_pages([], 2593342898, 'Users', 'pages_liked', # initialised just to feed the keyword argument
                #                    id_page_list) # the last argument is the important one

                info = get_user(user_id, api = api) # can get up to 800 user info
                user_info.append(info)
            # store user info
            cur.execute("UPDATE "+table_name+" SET description =?, location =?, status_count =?, follow_count =?, friend_count =?, changed_img =?, changed_theme =? WHERE user_id = ?",  user_info)
            '''
            get_user(user_ids, api = api, store = True)

In [ ]:
''' GET ALL FOLLOWERS OF A TWITTER PAGE, TOGETHER WITH INFO ON THEM '''
# slower than getting only ids, as it also has more stringent rate limits

twitter_page = 'Starbucks'

api = c_authenticate(c_auths, 0)
follw_count = api.get_user(twitter_page).followers_count 

get_followers_chained_all_stats(twitter_page, follw_count = follw_count , b_dir = b_dir) 

''' 
the above function is worth if one is working with few apps,
else it's better to just use get_followers or the function below'''

<a id ='faster_get_followers'> </a>
### Faster and smarter version

In [6]:
''' 
                FASTER VERSION
                
here we experimented a while loop.
We store only
before (eventually) sleeping, and at the end
'''

# modify this
twitter_page = 'bncalcio'

def store(twitter_page, num_page, bag):

def store(twitter_page, num_page, bag):
    
    if bag != []:      
        # if the page is new, create a value for the cursors in the db, else update it
        try:
            cur.execute("UPDATE Cursor2 SET cursor_value = " + str(num_page) + " WHERE twitter_page ='{}'".format(twitter_page))
        except: 
            cur.execute("INSERT INTO Cursor2 (twitter_page, cursor_value) VALUES (?,?)", [twitter_page, num_page])

        
        qst_marks = "?,"*len(bag[0])
        cur.executemany("INSERT INTO {} VALUES ({})".format(table_name, qst_marks[:-1]), bag) 
        bag = [] 
    con.commit()
    
    return bag

table_name = 'Users'
column_w_page_name = 'twitter_page'
column_with_duplicates = 'user_id'

cur.execute("CREATE TABLE IF NOT EXISTS Cursor2(twitter_page, cursor_value)")
# if the page is new, start from scratch, else from the cur val in the db
try:
    # will prompt the except if the page is new
    query_2 = "SELECT cursor_value FROM Cursor2 WHERE twitter_page = '{}'".format(twitter_page)              
    num_page = int(pd.read_sql_query(query_2, con).loc[0])
        
# if we have no ids or no cursor value, start from scratch
except:
    print("Starting from the first page")
    num_page = 0

app_num = len(c_auths) -1
bag = []
start_time = 2588440044 #to avoid sleeping at first iter
#s = set() # useful for debugging

# we will stop the call when we will reach last tweepy page
while True:
    
    # login to an app
    api = c_authenticate(c_auths, app_num)
    # useful for debugging
    #initial_s_len = len(s)

# change app and reset app if we passed through all,
# sleep if needed, store into the db the cursor
# and the pages retrieved
    app_num += 1
    if app_num == len(c_auths):
        app_num = 0
        bag = store(twitter_page, num_page, bag)
        
        time.sleep(max(0, 15*60 -  time.time() - start_time ))
        start_time = time.time()
    
# get follower ids and other info on them
    print("using app num", app_num)
    tweepy_cursor = tweepy.Cursor(api.followers, id= twitter_page, count = 200, cursor = num_page).pages(15)
    for result_set in tweepy_cursor:
        for user in result_set:
            bag.append([twitter_page, user.id, user.description, user.location, user.statuses_count, user.followers_count,
                      user.friends_count, user.default_profile_image, user.default_profile, np.nan])
            #s.add(user.id)    

    # update cursor value
    num_page = tweepy_cursor.next_cursor
    print("Cursor value = ", num_page)
    
    # when we reach last tweepy page,
    # it starts back from 0. Thus, if
    # we reach page 0, break the loop
    if num_page == 0 :
        break

# check if we really downloaded new data    
    #new_ids_downl = len(s) - initial_s_len
    #print("\nnew ids added: {}".format(new_ids_downl))

# we store the cursor coming before page 0.
# This way, if we call this function on a page whose
# followers we already have, it goes only for 1 iter
# before stopping (of course one could add a check to
# avoid wasting even that single app, but we found this
# to a more elegant solution)
store(twitter_page, tweepy_cursor.prev_cursor, bag)
print("\n Done")

using app num 0
1641896534143447276

new ids added: 3000
using app num 1
1609035824861973338

new ids added: 3000
using app num 2
1592754784274484941

new ids added: 3000
using app num 3
1579357836446908042

new ids added: 3000
using app num 4
1569673015652300793

new ids added: 3000
using app num 5
0

 Done


In [5]:
# to check that it worked
df = pd.read_sql_query("select * from Users where twitter_page ='{}'".format(twitter_page), con)

rem_duplicates_if(df, page_of_interest = twitter_page, 
                     column_w_page_name = column_w_page_name,
                     column_with_duplicates =  column_with_duplicates)

,twitter_page,user_id,description,location,status_count,follow_count,friend_count,changed_img,changed_theme,pages_liked
0,bncalcio,2185564980,Sabes que no dije mentiras 😉\n\n\n\n\n\n\n\n\n...,"Funafuti, Tuvalu 🇹🇻",9150.0,227.0,1660.0,0.0,1.0,None
1,bncalcio,1129430611058450434,Suivez l'actualité du football en temps réel s...,,519.0,22.0,305.0,0.0,1.0,None
2,bncalcio,725062262822723585,footballer,,60.0,123.0,2301.0,0.0,1.0,None
3,bncalcio,1063870559522615296,,,25.0,4.0,159.0,0.0,1.0,None
4,bncalcio,1014881241550778369,,,2696.0,27.0,793.0,0.0,1.0,None
...,...,...,...,...,...,...,...,...,...,...
16492,bncalcio,815182819676880896,Giornalista pubblicista iscritto all'Ordine de...,"Pisa, Toscana",1576.0,82.0,544.0,0.0,0.0,None
16493,bncalcio,2373917562,OttoChannel è visibile in tutta la Campania su...,"Campania, Italia",20524.0,834.0,25.0,0.0,0.0,None
16494,bncalcio,111658199,Antipatica e antifascista,"Benevento, Campania",1126.0,552.0,1244.0,0.0,0.0,None
16495,bncalcio,1511807881,"#WebManager, #WebDeveloper and much more... On...",Italia,160.0,259.0,131.0,0.0,1.0,None


<a id ='get_followers_ids'></a>

# Get Followers Ids

[Back on top](#top)

In [ ]:
# it was born as a standalone, single-user function, but it is pretty flexible, so, thanks
# to max_iters, it can be used aswell in a loop, while changing the app externally.
# One thing to note is that by storing in sql the tweepy page we reached, we can resume
# downloads without losing progress, if working across several days or in case of errors.
# This function aims at easiness of use and nice prints, we will show a more optimized function
# later
def get_followers_ids(twitter_page: str ,
                     column_with_duplicates =  'followerid',
                     column_w_page_name = 'brand',
                     table_name = 'twitter_data',
                     id_downl_at_step = 5000,
                    *max_iters 
                    ):
    '''
    Purpose:    Downloads ids of followers of a page and stores them in the db and table in use,
                inserting the page name in column_w_page_name, and ids in the column follower_id
                   
    Arguments:  twitter_page = page whose users ids we want to retrieve
                table_name = db table in which to store the data
                column_with_duplicates = column on which we sanity check to be sure we aren't 
                                         downloading duplicates ('followerid')
                column_w_page_name = column which contains the twitter pages names
                max_iters = if we want to limit the number of iterations
    '''
    # calculate how many ids we already have
    num_ids_downloaded = num_unique_followers(table_name, column_w_page_name, column_with_duplicates, twitter_page)
    # tot num of follw of the twitter page
    follw_count = api.get_user(twitter_page).followers_count   
    print("\n", twitter_page, "has {} followers, we already have {} ids".format(follw_count, num_ids_downloaded))
    
    # access/create the db containing the cursors values, will be useful 
    # to know at which tweepy page to start (or resume) the download
    cur.execute("CREATE TABLE IF NOT EXISTS Cursor(twitter_page, cursor_value)")  
    # if we already have some ids, start from the cur val in the db
    try:
        # if the page is new, it will give an error, prompting the except
        query_2 = "SELECT cursor_value FROM Cursor WHERE twitter_page = '{}'".format(twitter_page)              
        num_page = int(pd.read_sql_query(query_2, con).loc[0])
    # else start from scratch
    except:
        num_page = -1
        
        # if we already have all ids, just return
        # this will avoid being struck in a loop when we can't download
        # all users, for example because, since the first call,
        # new users started or stopped following the page
        if num_ids_downloaded > (follw_count*0.97):
            print("\nWe already have almost all ids ({} on {}). Interrupting the call".format(num_ids_downloaded,follw_count))
            return
        
        # a bit brutal, but, if we downloaded some ids without this function,
        # we might not have a cursor value and we wouldn't know from where to resume 
        # the download, meaning we must start from scratch, so let's clean at least.
        # Hopefully we always use this funct and don't incur in this scenario.
        # Avoid this scenario if we already have almost all ids
        elif num_ids_downloaded != 0:
            cur.execute("DELETE FROM {} where {} == '{}'".format(table_name, column_w_page_name, twitter_page))
            con.commit()
            num_ids_downloaded = 0
            print("Didn't know from where to resume download. Starting from scratch")

            
    # calculate num of iterations needed 
    if max_iters: #if we set a limit to the iterations
        # min to check if we need even less than the provided num of iters
        num_iterations = min(max_iters[0][0], int(np.ceil( (follw_count - num_ids_downloaded) / (id_downl_at_step * 15) )))
    else:
        # at each iter we get 15*id_downl_at_step ids, we want to do as many iters as
        # necessary to retrieve all ids we still have to retrieve
        num_iterations = int(np.ceil( (follw_count - num_ids_downloaded) / (id_downl_at_step * 15) ))
    print("I will go for {} iterations \n".format(num_iterations))   
    
    for iteration in range(0, num_iterations):
        
        print("We are at iter {}, on {}".format(iteration+1,num_iterations))
     
        # just to print more accurate informations
        start_page = (int(np.floor(num_ids_downloaded / id_downl_at_step)) - 1)
        # at each iteration we want to go through all tweepy pages 
        # necessary to get all ids (max 15 to avoid rate limits)
        num_of_pages_to_scroll =  min( 15, int(np.ceil( (follw_count - num_ids_downloaded) / (id_downl_at_step) )))

        for page in range(start_page, start_page + num_of_pages_to_scroll):
                        
            # 'page' and 'num_page' contain the same info, but 'num_pages' is in the
            # 'language' used by tweepy, that is, page 2 may be expressed as page 283942837
            # we will print user level readable information using 'page', while we will feed
            # 'num_pages' to tweepy
            print("I'm at page ", page)
            tweepy_cursor = tweepy.Cursor(api.followers_ids, id = twitter_page, cursor = num_page).pages(1) 
            lista_temp = []
            lista_temp.extend(tweepy_cursor)  
            
            # g contains the page name and the follower id, for each of the ids retrieved
            g = list(zip([twitter_page]*len(lista_temp[0]), lista_temp[0])) 
# can be made more efficient by storing outside the loop, or even the function,
# but, unless one generates a big number of apps, there are sleeping times anyways, 
# so it's worth to store each time we can
            cur.executemany("INSERT INTO {} VALUES (?,?)".format(table_name), (g)) 
            num_page = tweepy_cursor.next_cursor
            
        # check how many unique ids we have now, to see if things worked out well 
        new_num_ids = num_unique_followers(table_name, column_w_page_name, column_with_duplicates, twitter_page)
        
        # if the page is new, create a value for the tweepy_page in the db
        if num_ids_downloaded == 0 and iteration < 1: 
            cur.execute("INSERT INTO Cursor (twitter_page, cursor_value) VALUES (?,?)", [twitter_page, num_page])
        #  else we already have it, so just update it
        else: 
            cur.execute("UPDATE Cursor SET cursor_value = " + str(num_page) + " WHERE twitter_page ='{}'".format(twitter_page))

        con.commit()
        
        print('finished going through pages for this iteration\n')
        print("df final length = {}, \ndf initial length = {}, \nnew ids added: {}".format(new_num_ids, num_ids_downloaded, new_num_ids-num_ids_downloaded))
        print("\nnum pages visited at this iteration: {}\n".format(num_of_pages_to_scroll))
        
        if new_num_ids >= follw_count:
            print("Done. We have collected {} ids, {} currently has {} ids".format(new_num_ids, twitter_page, follw_count))
            print('--- breaking loop ---')
            break
        else:
            print('still going strong')
        
        # sleep 15 min except at the last iter. It won't sleep if we set max iters to 1, so 
        # we can call this function with max_iters = 1 while externally taking care of sleep
        # and app switch
        if iteration != num_iterations -1:          
            #get_followers(twitter_page = twitter_page)
            time.sleep(60*8)
            print("halfway back to work zzz \n")
            time.sleep(60*7)

# Return True if we already have all followers and we skipped the iterations. 
# We will use this bool when calling this function on a set of pages, to switch to the
# next page to download, in case we already have all ids for the current page and
# we bypassed the 'break' in the above iteration loop
    if num_iterations == 0:
        return True
    
    return False

In [20]:
''' SWITCHING TROUGH APPS, DOWNLOAD IDS OF FOLLOWERS OF ONE OR MORE PAGES '''

# need to change only twitter_pages of interest
column_with_duplicates =  'followerid'
column_w_page_name = 'brand'
table_name = 'twitter_data'

twitter_pages =  ["HandpressoNews", "MaisonValentino", "Prada", "Fendi", "PaneraiOfficial", "Azimut_Yachts","armani", "Ferrari","TiffanyAndCo","ROLEX","Bulgariofficial", "Lamborghini", "RivaYacht", "FerrettiGroup"] 
app_num = 0
executed_iterations = 0

for twitter_page in twitter_pages:
    print("downloading followers of", twitter_page)

    # login to the app and get the number of followers of the page
    api = c_authenticate(c_auths, app_num = app_num)
    follw_count = api.get_user(twitter_page).followers_count

    num_iterations = int( np.ceil(follw_count/75000) ) 
    
    for iteration in range(0, num_iterations):

        if app_num % len(c_auths) == 0:        
            start_time = time.time()
             
        print("using app num ", app_num)
        # call to the function we created
        bool_finished = get_followers_ids(twitter_page,
                                         column_with_duplicates,
                                         column_w_page_name,
                                         table_name,
                                         5000,
                                         (1,) )
        
        # break the inner loop if we already have all followers for this page
        if bool_finished == True:
            break
        
        # each iter calls 15 times the tweepy_function, so at each iter we change app
        app_num += 1   

# every time we passed through all the apps, we make a time check and reset app_num
        if app_num % len(c_auths) == 0:
            sleep_time = 60*15 - (time.time() - start_time) 
            print("\n sleeping for {} \n".format(sleep_time))
            time.sleep(max(0, sleep_time))
            app_num = 0
        
        api = c_authenticate(c_auths, app_num) 

downloading followers of OliviaPalermo
using app num  0

 OliviaPalermo has 418414 followers, we already have 75001 ids
I will go for 1 iterations 

We are at iter 1, on 1
I'm at page  14
I'm at page  15
I'm at page  16
I'm at page  17
I'm at page  18
I'm at page  19
I'm at page  20
I'm at page  21
I'm at page  22
I'm at page  23
I'm at page  24
I'm at page  25
I'm at page  26
I'm at page  27
I'm at page  28
finished going through pages for this iteration

df final length = 150001, 
df initial length = 75001, 
new ids added: 75000

num pages visited at this iteration: 15

still going strong
using app num  1

 OliviaPalermo has 418414 followers, we already have 150001 ids
I will go for 1 iterations 

We are at iter 1, on 1
I'm at page  29
I'm at page  30
I'm at page  31
I'm at page  32
I'm at page  33
I'm at page  34
I'm at page  35
I'm at page  36
I'm at page  37
I'm at page  38
I'm at page  39
I'm at page  40
I'm at page  41
I'm at page  42
I'm at page  43
finished going through pages

<a id='faster'> </a>
## Faster version
[Back on top](#top)

In [34]:
# Since evaluation may be on code efficiency as well, we created this function just to show 
# we are able to create faster functions, even though we won't usually need to use them.
# To be faster, here we will assume that we start from 0 ids for the twitter pages we
# give as an argument, just need a careful usage in order not to waste time downloading 
# what we already have
def main_get_ids_fast(twitter_pages,
                      c_auths):  
    '''
    Purpose: manage apps, pages and iterations before calling
             the function to get follower ids of a page,
             for each page in twitter_pages
    '''
    # start from the first app
    app_num = 0
    pag_rem = 0
    start_time = time.time()
    
# each call of this loop allows to download ids of all followers of a twitter page
    for twitter_page in twitter_pages:   
        
        # initialize the tweepy cursor to start from scratch
        cursor_val = -1
        # login to an api just to get tot num of follw of the twitter page
        # an alternative would be to just keep downloading until we
        # download less than 70k new ids
        api = c_authenticate(c_auths, app_num = app_num)
        follw_count = api.get_user(twitter_page).followers_count        
# each app downloads 75k follw, calculate how many times we need to iterate among them
        num_calls =int (np.ceil ( follw_count/75000))

# Skipped at first iteration.     
# If we have still a decent amount of pages to use after we're done downloading ids from a page,
# stay with the app we just used and take advantage of the other pages we can use.
# Expecially useful if we have twitter pages with low values of follw_count,
# as, without this step, it would mean to waste a lot of app calls
# just to download a couple of pages of results. Otherwise, one could remove this 'if' to
# make things faster
        if pag_rem > 2:
            app_num -=1 
            num_pages_to_scroll = pag_rem
            #print("Switched back to app num {}, still have {} pages".format(app_num, pag_rem))
        else: 
            num_pages_to_scroll = 15
        print("calling get_ids with app", app_num)
        # download all followers of this page, going through the num_call we calculated    
        app_num, pag_rem = call_get_ids(cursor_val, num_pages_to_scroll, start_time, app_num, num_calls, c_auths, follw_count, twitter_page)
        print("\nDownloaded this page followers. Now app_num is", app_num)
        
# to further optimize, one could store results only halfway through and on last iter,
# instead of storing each time we downloaded all the followers of a page.
# The following works, but we should also return g in call_get_ids, and append it to
# list_of_lists, on top of removing the storing part in call get_ids. Thus this is
# just a pseudo-code.
# Dividing by 4 also works
        # i += 1
        # if int( i% (len(twitter_pages)/2)) == 0 and i != 0:
        #    cur.executemany("INSERT INTO twitter_data VALUES (?,?)", (list_of_lists)) 
        #    con.commit()  
        #    list_of_lists= []
            
    return
 
        
def call_get_ids(cursor_val, num_pages_to_scroll, start_time, app_num, num_calls, c_auths,follw_count, twitter_page ):
    '''
    Purpose: get all follower ids of a twitter_page, managing the
             rate limits and storing results at the end
    '''
# we may be using an app which we already used. In this case, if we will use
# only this app, aka one call is enough, we will want to return, as number of
# pages remaining for the last app used, not 15 - pages_we_will_use, but, instead,
# pages_we_had_left - pages_we_will_use, in order not to overestimate our app capability
    if num_calls == 1:
        max_pages = num_pages_to_scroll
    else:
        max_pages = 15
        
    page_list = []
    for call in range(app_num, num_calls):
        print("i'm at call {}, with app {}".format(call,app_num))
        # authenticate to app number app_num
        api = c_authenticate(c_auths, app_num = app_num)
        
# Calculate if we need less than 15 -or whatever we have left- pages for last iteration;
# as above, one may delete this to speed_up if only downloading big datasets.
        if call ==  num_calls - 1:
            # we already have 75000*num_calls -2 ids
            num_pages_to_scroll = min(max_pages, int(np.ceil( (follw_count - (75000*call))/5000 )))
            pages_used_last_iter = num_pages_to_scroll
            
        #print("going for num pages", num_pages_to_scroll)
        # download 5k * num_pages_to_scroll ids
        tweepy_cursor = tweepy.Cursor(api.followers_ids, id = twitter_page, cursor = cursor_val, count = 5000).pages(num_pages_to_scroll) 
        
        # put them into a list
        for tweepy_page in tweepy_cursor:
            page_list.extend(tweepy_page) 
        # update the cursor value
        cursor_val = tweepy_cursor.next_cursor
        
        # change app_num
        app_num, start_time, num_pages_to_scroll = change_app(app_num, start_time, c_auths)
        
        
    # store in the db the followers of this twitter page
    g = list(zip([twitter_page]*len(page_list), page_list)) 
    print("storing this number of users:", len(g))
    cur.executemany("INSERT INTO twitter_data VALUES (?,?)", (g)) 
    con.commit()  
                                  
    return app_num, max_pages - pages_used_last_iter #g


def change_app(app_num, start_time, c_auths):
    '''
    Purpose: update app_num and check if we need to reset it or to sleep
    ''' 
    app_num +=1
    # since we are switching app, we can reset num_pages_to_scroll
    num_pages_to_scroll = 15
    if app_num == len(c_auths):   
        # with enough apps it can sleep for 0 secs
        sleep_time = max(0, (60*15 + 4) - (time.time() - start_time))
        print("sleeping for: ", sleep_time)
        time.sleep(sleep_time)
        start_time = time.time()
        app_num = 0
    
    print("switched to app", app_num)
    return app_num, start_time, num_pages_to_scroll

In [36]:
test = main_get_ids_fast( ['selutbooks', 'bncalcio'], c_auths)

calling get_ids with app 0
i'm at call 0, with app 0
switched to app 1
storing this number of users: 2901

Downloaded this page followers. Now app_num is 1
calling get_ids with app 0
i'm at call 0, with app 0
switched to app 1
storing this number of users: 16531

Downloaded this page followers. Now app_num is 1


<a id ='get_pages'></a>

# Get pages

Note: 
- we store only pages above a certain threshold of followers, to avoid retrieving what we deem un-interesting information, like an user's actual friends
- to get the pages liked by the followers of a Twitter page, we need to have at least their ids in the table Users, the best way to achieve this is by calling get_followers on such page, otherwise the faster way is to call get_followers_ids by accordingly changing the storing part (by default it stores in the twitter_data table)


[Back on top](#top)

In [3]:
database_to_use = 'Brand_Followers(backup).db'

# connect to the db
con, cur =     connect_to_sql(database_name = database_to_use, 
                              return_df = False)  

In [6]:
# need to change only twitter_page of interest
twitter_page = 'lavazzagroup'

# if you use twitter_data table, need to slightly change sql code 
table = 'Users'                  # or twitter_data
column_w_pages = 'twitter_page'  # or brand
id_page_cols = 'user_id, pages_liked'         # or followerid

df = pd.read_sql_query("SELECT {} FROM '{}' WHERE twitter_page == '{}'".format(id_page_cols, table, twitter_page), con)

# create a list of users for which we still have
# to retrieve pages liked
user_ids = df[df['pages_liked'] == '']["user_id"].to_numpy()
#user_ids = df[df[id_page_cols.split()[1]].isnull()][id_page_cols.split()[0][:-1]].to_numpy()

print(len(np.unique(user_ids))) # how many pages_liked we still have to retrieve
user_ids 

7


array([955908060215676938, 975789565016453120,         2317364652,
               2462928132,          195368541,         1849299108,
                947908681], dtype=int64)

In [ ]:
'''
some users may have gotten blocked by Twitter or may have deleted their account. 
In this case, we can't even retrieve their number of friends;
other users (a lot of them) have private accounts, so we can see how many friends they
have, but we can't download them.
The best way we found to deal with these is to use a try-except in the first case, and
a user_method to check for the second case. Using try-except also in the second case
leads to a lot of complications. Indeed, for example, calling api.friends inside a try
will still count as a call, so, after 15 such calls, one should switch app, having 
wasted one app without downloading anything. User methods, instead, have much higher
rate limits
'''

def get_pages(_id, api, threshold = 2500, ret_cur = False, cursor = -1, num_pages = 1):
    '''
    WHAT DOES IT DO?
    Given a single id, it returns a list of the pages followed by that user, given that
    they have at least 2500 followers.

    CAUTION:
    - It does not store them into a db! To do that, create a list [get_pages(_id), _id] for
      a number of ids, and feed it to store_pages(multiple_users = list )
    - Rate limit after 15 calls, no checks inside the function, since it's called
      on a single id, so be careful
    '''
    #print("received id:", _id)
    #try:
    page_list = []
    # generally, users can't have more than 5k pages followed. We can only retrieve up to 200 per call
    tweepy_cursor = tweepy.Cursor(api.friends, count = 200, id= _id, cursor = cursor).pages(num_pages) 
    for objects in tweepy_cursor:
        for page in objects:
            # threshold to retrieve only relevant pages followed
            if page.followers_count > threshold: 
                page_list.extend([page.screen_name])
    #except:
    #    print("Private profile, no pages retrieved for this user, returning an empty list")
    #    page_list = []

    if ret_cur == False:
        return page_list
    
    else:
        return page_list, tweepy_cursor.next_cursor
    
    
def store_pages(page_list = [], _id = 2593342898, 
                table_name = 'Users', col_w_pages = 'pages_liked', 
                *multiple_users): # give multiple users as [(_id, page_list),(_id, page_list)...]
    '''
    WHAT DOES IT DO?
    Stores into the db the pages liked by an user, or by more users if multiple_users is used.

    HOW ARE THE PAGES STORED?
    Since we can store only a single value in the db, either we create a row for each page 
    followed, for each user, or we compress this info, separating each element with an _***_. 
    We have chosen the latter option
    '''
    # store the pages into a column of the db, after joining them into
    # a list and separating them with a token

    if not multiple_users:
        string_pag_seguite = ""
        for pagina in page_list: 
            string_pag_seguite +=  pagina + " _***_ "

        cur.execute("UPDATE "+table_name+" SET " + col_w_pages + " = ('{}') WHERE user_id = ".format(string_pag_seguite[:-7]) + str(_id) )
        con.commit()

    else:
        bag = []
        for user in multiple_users[0]: # [0] because it's a karg, so it envelopes all in a tuple
            _id =  user[0]
            page_list = user[1]
            string_pag_seguite = ""

            for pagina in page_list:
                #print("id = ", _id)
                #print("pagina = ", pagina)
                string_pag_seguite +=  pagina + " _***_ "
            bag.append((string_pag_seguite[:-7], _id))
        
        print("table = ", table_name, "storing into col:", col_w_pages)
        #print("Storing into sql the following list", bag)
        cur.executemany("UPDATE "+table_name+" SET " + col_w_pages + " = ? WHERE user_id = ?",  bag)
        con.commit()

    return


def app_num_check(c_auths, app_num, start_time):
    '''
    Purpose: if we switched through all the apps, sleep and reset app_num index
    '''
    # nap check, app_num reset
    if app_num == len(c_auths):    
        sleep_time = max(0, (60*15 + 4) - (time.time() - start_time))
        print("\nSleeping for ", sleep_time,"\n")
        time.sleep(sleep_time)
        start_time = (time.time())
        app_num = 0
        
    # else don't change app_num and don't sleep                  
    return app_num, start_time


def change_and_download(c_auths, app_num, rem_pages, user_id, cursor, pages, start_time):
    '''
    Purpose: It is called when we need to change app.
             Authentichate to the app in position app_num and download the necessary pages,
             up to 15. 
             If more than 15 pages are needed, it recursively calls itself, upon
             updating app_num and calling app_num_check, downloading all pages of that
             user (below we limit ourselves to retrieving
             a maximum of 4k pages per user, to avoid wasting time downloading even 20k pages
             for a single user, they wouldn't be representative of an actual interest)
    '''
    api = c_authenticate(c_auths, app_num)
    print("switching to app number:",app_num)

    rem_pages = abs(rem_pages)
    if rem_pages > 15:

        print("changing one more app for this user")
        # download the maximum with the new app, then change app and download the rest
        pages_app_switch, cursor = get_pages(_id = user_id, api = api, num_pages = 15, cursor = cursor,  ret_cur = True)
        pages.extend(pages_app_switch)
        # update the remaining num of pages to download
        rem_pages -= 15
        # change app
        app_num += 1
        # check if we reached the last app, needing an index reset and a nap
        app_num, start_time = app_num_check(c_auths, app_num, start_time)
        # re-call the function to retrieve the rest, using the next app
        page_limit, app_num, api, start_time = change_and_download(c_auths, app_num, rem_pages, user_id, cursor, pages, start_time)

        return page_limit, app_num, api, start_time


    # download the remaining pages for this user
 
    pages_app_switch = get_pages(_id = user_id, api = api, num_pages = rem_pages, cursor = cursor)
    pages.extend(pages_app_switch)
    print("downloaded all liked pages above a threshold of followers ({}) for this user ({})".format(len(pages), user_id))
    
    # count how many pages we can download with this app before rate limit
    page_limit = 15 - rem_pages

    return page_limit, app_num, api, start_time

# here we use the above defined functions in a loop with conditions.

# Estimate an upward biased number for how many iterations we will need
num_iterations = int(np.ceil( len(user_ids) / (15 * len(c_auths)) ))
app_num = 0
page_limit = 15

# initializing app and time count
api = c_authenticate(c_auths, 0)
start_time = time.time()

for user_id in user_ids:
    
# calculate number of pages to download, check if the account
# is blocked using a try-except
    try:
        print("page limit (using app {}) =".format(app_num), page_limit)

        # min to avoid wasting much time on users with absurd amounts of followed pages
        num_friends = min (4000, api.get_user(id= user_id).friends_count)
        # number of tweepy pages to download
        num_pages = int ( np.ceil( num_friends / 200 ))
        if num_pages == 0:
            print("This user has a temporarily limited account, skipping it")
            continue
        # rem_pages is the num of tweepy pages we will have left on this app
        rem_pages = page_limit - num_pages
        #print("remaining pages:", rem_pages)

    except:
        # go to the next user
        print("Skipping this user (blocked or deleted profile)")
        # this way we won't check for him again, we will consider he only
        # follows the page we know he follows
        store_pages([twitter_page], user_id, table, id_page_cols.split()[1])
        continue
        
# if he has private profile, skip him
    private_profile = api.get_user(user_id).protected
    if private_profile == True:
        # this way we won't check for him again
        store_pages([twitter_page], user_id, table, id_page_cols.split()[1])
        print("Skipping this user: he has a private profile. His id: {}".format(user_id))
        continue
 

# we still consider a final broad try-except statement, as some rare
# situations can pass through the above screening, for example some
# users no longer have an account, but we can still retrieve
# how many friends they have and whether their account was private.
# This happened, for example, with user_id = 105179723
    try:

# Now we've done all the checks, and we know how many pages he follows.
# We only need to manage the rate limits


    # if we can, download and store the pages
        if rem_pages >= 0:
            print("downloading all the liked pages above a threshold of followers ({}) for this user ({})".format(num_friends, user_id))
            # update the num of tweepy pages we can still downl with this app
            page_limit = rem_pages
            pages = get_pages(_id = user_id, api = api, num_pages = num_pages)
            # if one uses a really big number of apps, the storing part can be put
            # right before sleeping, calling the below function with its keyword arg
            store_pages(pages, user_id, table, id_page_cols.split()[1]) 
            #print("done, next user")



    # if we would reach the rate limit: download what we can,
    # switch app, check that we switched apps no more than len(c_auths)
    # without resetting and sleeping, download the rest of the pages
        else:
            print("\ni would reach the rate limit if i downloaded all pages ({}) for this user ({})".format(num_friends, user_id))

            # download what you can with the current app
            if page_limit > 0:
                print("downloading what i can with the current app")
                pages, cursor = get_pages(_id = user_id, api = api, num_pages = page_limit, ret_cur = True)
            else: 
                pages = []
                cursor = -1

            # update the app number
            app_num += 1

# check if we passed through all apps and we need to sleep and reset the index
            app_num, start_time = app_num_check(c_auths, app_num, start_time)

# login to the new app, download the remaining pages and get the num of pages we can still downl
            page_limit, app_num, api, start_time = change_and_download(c_auths, app_num, rem_pages, user_id, cursor, pages, start_time)

# store pages in the db 
            store_pages(pages, user_id, table, id_page_cols.split()[1]) 

            # check that it stored the results (number should go down)
            #df = pd.read_sql_query("SELECT {} FROM '{}' WHERE twitter_page == '{}'".format(id_page_cols, table, twitter_page), con)
            #print(len(df[df[id_page_cols.split()[1]].isnull()][id_page_cols.split()[0][:-1]].to_numpy()))
   
    except:
        print("This user doesn't exist anymore")
        
print("\nDone")

# one can use https://tweeterid.com/ to doublecheck that we got an exception because
# the profile is indeed private

# in the output printed we reached the rate limit
# because we restarted the notebook and called the
# function again, without waiting 15 mins

<a id ='dataset'></a>

# Dataset Operations

[Back on top](#top)

In [49]:
''' Cleaning '''
# Check the followers we downloaded 
table = 'Users'    # Users contains detailed info, twitter_data only contains ids by page followed
column_w_pages = 'twitter_page' #  twitter_page --- brand
column_w_ids = 'user_id'        #  user_id --- followerid

df = pd.read_sql_query("SELECT * FROM '{}'" .format(table), con) 
# remove duplicates in the whole dataset
df = rem_duplicates_dataset(df, column_w_page_name = column_w_pages, column_with_duplicates= column_w_ids)

# update the table, letting in it only non-duplicated values
#df.drop(columns = "index", inplace = True)
#df.to_sql(name = table, if_exists = 'replace', con = con, index = False)

df

Initial shape (13991, 10)
Final shape (13991, 10)


,twitter_page,user_id,description,location,status_count,follow_count,friend_count,changed_img,changed_theme,pages_liked
0,lavazzagroup,38976229,None,None,NaN,NaN,NaN,NaN,NaN,50cent _***_ Usher _***_ johnlegend _***_ Tige...
1,lavazzagroup,1037013378559107074,None,None,NaN,NaN,NaN,NaN,NaN,nataliehemby _***_ DukeNews _***_ DukeMBB _***...
2,lavazzagroup,22661036,None,None,NaN,NaN,NaN,NaN,NaN,WaffleHouse _***_ Anime _***_ USPSHelp _***_ U...
3,lavazzagroup,21751873,None,None,NaN,NaN,NaN,NaN,NaN,VirtualAstro _***_ GaryBarlow _***_ cyprusmail...
4,lavazzagroup,2468860608,None,None,NaN,NaN,NaN,NaN,NaN,JGrobicki _***_ TineGoetzsche _***_ swiftlyama...
...,...,...,...,...,...,...,...,...,...,...
13986,lavazzagroup,28196164,Marketing & Innovation Director @wearesocialit...,"Milan, Italy",24929.0,5391.0,744.0,0.0,0.0,BillGates _***_ istsupsan _***_ HowellONeill _...
13987,lavazzagroup,2216552828,I think life is beautiful. As long as you have...,,640.0,233.0,297.0,0.0,0.0,Tokyo2020 _***_ spelacchio _***_ ItalianCommen...
13988,lavazzagroup,226117937,,Milano,1051.0,468.0,184.0,0.0,1.0,AirAsiaSupport _***_ stephenfry _***_ rustyroc...
13989,lavazzagroup,41348521,Group Account Director @ We Are Social Italia....,Milan - Italy,260.0,208.0,157.0,0.0,0.0,wearesocial _***_ salvadorcunha _***_ adidasor...


In [24]:
# sanity check: lavazza should have around 14k followers
page_of_interest = 'lavazzagroup'
len(df[df[column_w_pages] == page_of_interest])

13991

In [27]:
''' see the pages from which we collected data '''
df[column_w_pages].unique()

array(['MonsterEnergy', 'MonsterGaming', 'Nespresso', 'Starbucks',
       'lavazzagroup', 'Bulgariofficial', 'letortedirenato',
       'CakeDesignIta', 'ricettedolci1', 'DolciRicette',
       'Dulcisssinforno', 'DolciSemplici', 'LeTortine', 'OXO',
       'HydroFlask', 'swellbottle', 'YETICoolers', 'MonsterEnergyIT',
       'redbullITA', 'burn_ITA', 'ROLEX', 'TiffanyAndCo', 'Ferrari',
       'ChiaraFerragni', 'camihawke', 'marianodivaio', 'sferaebbasta',
       'saluis97', 'aperolspritzita', 'CampariItalia', 'BACARDI_IT',
       'HandpressoNews', 'MaisonValentino', 'Prada', 'Fendi',
       'PaneraiOfficial', 'Azimut_Yachts', 'armani', 'RivaYacht',
       'FerrettiGroup', 'GoPro', 'ray_ban', 'hamiltonwatch',
       'GlycineWatch', 'TISSOT', 'GentryWatch', 'LandRover',
       'LandRoverItalia', 'style_italia', 'VanityFairIt', 'esquireitalia',
       'esquire', 'BoggiMilano', 'theonlyparmesan', 'ideericette',
       'illyIT', 'CaffeVergnano', 'Kimbo_It', 'LavazzaUK', 'Lamborghini',
       